# AutoML

In [1]:
pip install autokeras

In [2]:
import numpy as np
import tensorflow as tf
import autokeras as ak
from tensorflow.python.keras.utils.data_utils import Sequence

## 모델 정의, 설정, 학습

In [3]:
# Data Preprossing
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape)  # (60000, 28, 28)
print(y_train.shape)  # (60000,)
print(y_train[:3])  # array([7, 2, 1], dtype=uint8)

(60000, 28, 28)
(60000,)
[5 0 4]


In [4]:
# Initialize the image classifier.
clf = ak.ImageClassifier(
    overwrite=True,
    max_trials=1)

# Feed the image classifier with training data.
clf.fit(x_train, y_train, validation_split=0.20, epochs=10)

# Export as a Keras Model.
model = clf.export_model()

print(type(model))  # <class 'tensorflow.python.keras.engine.training.Model'>

try:
    model.save("model_autokeras", save_format="tf")
except:
    model.save("model_autokeras.h5")

Trial 1 Complete [00h 01m 06s]
val_loss: 0.04281419515609741

Best val_loss So Far: 0.04281419515609741
Total elapsed time: 00h 01m 06s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
1875/1875 [==============================] - 7s 3ms/step - loss: 0.3035 - accuracy: 0.9079
Epoch 2/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0787 - accuracy: 0.9753
Epoch 3/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0622 - accuracy: 0.9815
Epoch 4/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0491 - accuracy: 0.9844
Epoch 5/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0481 - accuracy: 0.9848
Epoch 6/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0407 - accuracy: 0.9871
Epoch 7/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0356 - accuracy: 0.9886
Epoch 8/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0371 - accuracy: 0.9886
Epoch 9/10

In [5]:
# Load Data from Disk
from tensorflow.keras.models import load_model

loaded_model = load_model("model_autokeras", custom_objects=ak.CUSTOM_OBJECTS)
loaded_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28)]          0         
_________________________________________________________________
cast_to_float32 (CastToFloat (None, 28, 28)            0         
_________________________________________________________________
expand_last_dim (ExpandLastD (None, 28, 28, 1)         0         
_________________________________________________________________
normalization (Normalization (None, 28, 28, 1)         3         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0     

In [6]:
# Predict with the best model.
predicted_y = loaded_model.predict(tf.expand_dims(x_test, -1))
print(predicted_y)

[[2.93125116e-12 3.47398464e-11 1.63623259e-09 ... 9.99999881e-01
  1.11967770e-11 1.01627293e-07]
 [3.95244768e-07 4.26138541e-07 9.99752581e-01 ... 8.46146558e-13
  2.05773034e-08 6.01013380e-13]
 [1.53064916e-09 9.99993563e-01 4.88191006e-07 ... 1.39847521e-06
  1.29182169e-07 8.38622616e-10]
 ...
 [1.10762083e-15 6.89383099e-12 5.01681987e-15 ... 1.11938858e-08
  3.96266813e-07 2.28074093e-08]
 [7.82238718e-09 4.80170509e-14 3.11079912e-10 ... 7.15171585e-12
  2.69121756e-06 7.98825228e-10]
 [1.10899085e-08 1.25097287e-17 5.91147087e-09 ... 1.36871538e-16
  3.56405998e-08 5.24068289e-12]]


In [10]:
# Evaluate the best model with testing data.
print(clf.evaluate(x_test, y_test))

313/313 [==============================] - 1s 3ms/step - loss: 0.0307 - accuracy: 0.9902
[0.0307270810008049, 0.9901999831199646]
